Run `setup.ipynb` the first time - create fresh table - do not run setup in an automated job

In [0]:
from aeso_api import aeso
from datetime import datetime, timedelta
from daft.unity_catalog import UnityCatalog
import pytz
import daft

# point Daft to your UC server
unity = UnityCatalog(
    endpoint="https://adb-984752964297111.11.azuredatabricks.net/",
    token=(
        dbutils.notebook.entry_point.getDbutils()
        .notebook()
        .getContext()
        .apiToken()
        .getOrElse(None)
    ),
)

# load UC table - empty after running setup notebook
unity_table = unity.load_table("dhurley_catalog.benchmarks.aeso_pool_prices")

In [0]:
# AESO API key and instance initialization
aeso_api_key = 'b3b431c1c956490580df23f395786ba6'
aeso_instance = aeso(aeso_api_key)

# Get Current Date, and Current Date + 1
utc_time = pytz.utc.localize(datetime.utcnow())
calgary_time = utc_time.astimezone(pytz.timezone("America/Edmonton"))

start_date = (calgary_time - timedelta(hours=1)).strftime("%Y-%m-%d")
end_date = (calgary_time).strftime("%Y-%m-%d")

# Fetch pool price report from AESO API
resp = aeso_instance.get_pool_price_report(
    start_date=start_date, end_date=end_date
)

In [0]:
# Format AESO API response to dictionary
data = {
    "date": [str(p.begin_datetime_utc) for p in resp if p.pool_price is not None],
    "poolPrice": [str(p.pool_price) for p in resp if p.pool_price is not None]
}

# Write to delta table in UC
daft_df = daft.from_pydict(data)
daft_df.write_deltalake(unity_table, mode='append')

In [0]:
import dbdemos
dbdemos.install('uc-04-system-tables', catalog='main', schema='billing_forecast')

In [0]:
%pip install dbdemos

